This shows how can data can be extracted from Sepsis_imp.csv and processed to be passed as inputs of a neural network

In [1]:
import pandas as pd
from parse_dataset import *
from tqdm import tqdm

Load and parse

In [3]:
sum([None,True,True])

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

In [2]:
data = pd.read_csv('../Sepsis_imp.csv')

In [3]:
replace_absurd_temperatures(data)
data = drop_patients_with_absurd_weights(data)
data = drop_patients_with_unrealistic_HR_or_BP(data)
data = add_relative_time_column(data)
data = drop_patient_with_negative_input(data)
add_small_quantities(data)
create_action_column(data)

In [7]:
from sklearn.preprocessing import StandardScaler
log_scaler = StandardScaler()
scaler = StandardScaler()
train_idx, test_idx = split_train_test_idx(data)

# scale on train data only
scaler.fit(data.loc[data.icustayid.isin(train_idx)][numerical_columns_not_to_be_logged])
log_scaler.fit(np.log(data.loc[data.icustayid.isin(train_idx)][numerical_columns_to_be_logged]))

StandardScaler(copy=True, with_mean=True, with_std=True)

Matrify

In [16]:
binary_columns

['gender', 're_admission', 'rrt', 'mechvent', 'sedation']

In [15]:
n_cols = len(binary_columns)+len(numerical_columns_not_to_be_logged)+len(numerical_columns_to_be_logged)
n_train = train_idx.shape[0]
X_bin_train = np.zeros((n_train, 32, len(binary_columns)))
X_num_train = np.zeros((n_train, 32, n_cols-len(binary_columns)))
X_action_train = np.zeros((n_train, 32, 2))
for k, (idx, df) in tqdm(enumerate(data.loc[data.icustayid.isin(train_idx)].groupby('icustayid'))):
    T = len(df)
    x_bin = df[binary_columns].values
    # scale these
    x_num_not_logged = scaler.transform(df[numerical_columns_not_to_be_logged])
    x_num_logged = log_scaler.transform(np.log(df[numerical_columns_to_be_logged]))
    x_num = np.concatenate([x_num_not_logged, x_num_logged], -1)
    actions = df[action_cols].values
    
    # pad with zeros
    x_bin = np.concatenate([x_bin, np.zeros((32-T, len(binary_columns)))], 0)
    x_num = np.concatenate([x_num, np.zeros((32-T, n_cols-len(binary_columns)))], 0)
    actions = np.concatenate([actions, np.zeros((32-T, 2))], 0)
    
    # k-th history
    X_bin_train[k] = x_bin
    X_num_train[k] = x_num
    X_action_train[k] = actions

18100it [00:34, 526.50it/s]
